In [1]:
import numpy as np
import pandas as pd
import math
from sklearn.metrics import r2_score
import methods 
import metrics 
from metrics import mse
from methods import SGD, RMSprop, RMSprop_regul, Adagrad, Adam

In [2]:
def predict(X,optim): 
    return np.dot(X, optim[:-1]) + optim[-1]  

In [3]:
def k_fold_cross_validation(X, Y, method, reg_type, k=5):
    size_test = len(X) // k
    begin = 0
    end = size_test + len(X) % k
    m_mse = 0
    m_r2 = 0
    for i in range(k):
        w = np.zeros(len(X.columns) + 1)
        
        X_train = pd.concat([X[:begin], X[end:]])
        Y_train = pd.concat([Y[:begin], Y[end:]])
        X_test = X[begin:end]
        Y_test = Y[begin:end]

        if method == 'SGD':
            if reg_type == 'No':
                optim, err = SGD(X, Y,w,reg_type,**paramSGD)
            elif reg_type == 'L1':
                optim, err = SGD(X, Y,w,reg_type,**paramSGDL1)
            elif reg_type == 'L2':
                optim, err = SGD(X, Y,w,reg_type,**paramSGDL2)
        elif method == 'RMSprop':
              optim, err = RMSprop(X, Y,w,**paramRMS)
        elif method == 'RMSprop_regul':
            if reg_type == 'L1':
                optim, err = RMSprop_regul(X, Y,w,reg_type,**paramRMSL1)
            elif reg_type == 'L2':
                optim, err = RMSprop_regul(X, Y,w,reg_type,**paramRMSL2)       
        elif method == 'Adagrad':
            if reg_type == 'No':
                optim, err = Adagrad(X, Y,w,reg_type, 0) 
            elif reg_type == 'L1':
                optim, err = Adagrad(X, Y,w,reg_type, **paramAdaL1)
            elif reg_type == 'L2':    
                optim, err = Adagrad(X, Y,w,reg_type, **paramAdaL2)
        elif method == 'Adam':
            if reg_type == 'No':
                optim, err = Adam(X, Y,w,reg_type, 0) 
            elif reg_type == 'L1':
                optim, err = Adam(X, Y,w,reg_type, **paramAdamL1)
            elif reg_type == 'L2':    
                optim, err = Adam(X, Y,w,reg_type, **paramAdamL2)
            
        Y_p = predict(X_test,optim)

        a_mse = mse(Y_test,Y_p,begin)
        a_r2 = r2_score(Y_test,Y_p)
        m_mse += a_mse
        m_r2 += a_r2
        print(f"Step {i+1}: MSE = {a_mse}, R2 = {a_r2}")

        begin += size_test
        end += size_test

    print(f"Average MSE: {m_mse / k}")
    print(f"Average R2: {m_r2 / k}")

## Результаты

Используем заранее подобранные параметры

In [9]:
#настроенные параметры
paramSGD= {'reg_param': 0,'coef_grad': 0.01,'coef_prev_grad': 0.001}
paramSGDL1= {'reg_param': 10,'coef_grad':  0.01,'coef_prev_grad': 0.001}
paramSGDL2= {'reg_param': 100, 'coef_grad': 0.01,'coef_prev_grad': 0.001}
paramRMS = {'gamma': 0.8, 'num': 80}
paramRMSL1 = {'reg_param': 50, 'gamma': 0.8, 'num': 50}
paramRMSL2 = {'reg_param': 50, 'gamma': 0.8, 'num': 50}
paramAdaL1 = {'reg_param': 10000}
paramAdaL2 = {'reg_param': 10}
paramAdamL1 = {'reg_param': 10}
paramAdamL2 = {'reg_param': 10}

Считываем тренировочные данные

In [14]:
df=pd.read_csv('train.csv', delimiter=',') 
df=df.drop(df.columns[[0]], axis='columns')
X = df.drop(columns=["DEP_DELAY"])
Y = df["DEP_DELAY"]

**SGD**

Результаты для алгоритма стохастического градиентного спуска без регуляризации, с L1 и L2 регуляризациями

In [95]:
method = 'SGD'
reg_type = 'No'
k_fold_cross_validation(X, Y, method, reg_type)

Step 1: MSE = 1583.1979658852629, R2 = 0.009512361381599721
Step 2: MSE = 1552.6829928842717, R2 = 0.008916195042106945
Step 3: MSE = 1571.3292168913358, R2 = 0.008504066652072373
Step 4: MSE = 1591.9803444247832, R2 = 0.009073354511640597
Step 5: MSE = 1423.7521636702702, R2 = 0.010649659066893191
Average MSE: 1544.5885367511846
Average R2: 0.009331127330862565


In [96]:
method = 'SGD'
reg_type = 'L1'
k_fold_cross_validation(X, Y, method, reg_type)

Step 1: MSE = 1668.6016742177155, R2 = -0.04391830188239232
Step 2: MSE = 1636.3373799602793, R2 = -0.04448073699401367
Step 3: MSE = 1656.1371128281228, R2 = -0.0450090247060102
Step 4: MSE = 1679.1195136359759, R2 = -0.04516633817018456
Step 5: MSE = 1507.1245108451906, R2 = -0.047284904410262785
Average MSE: 1629.4640382974567
Average R2: -0.045171861232572705


In [97]:
method = 'SGD'
reg_type = 'L2'
k_fold_cross_validation(X, Y, method, reg_type)

Step 1: MSE = 1668.6016734844684, R2 = -0.04391830142365749
Step 2: MSE = 1636.3373794191057, R2 = -0.04448073664862129
Step 3: MSE = 1656.1371123105705, R2 = -0.0450090243794119
Step 4: MSE = 1679.1195130637227, R2 = -0.04516633781401347
Step 5: MSE = 1507.124510237127, R2 = -0.04728490398774876
Average MSE: 1629.4640377029987
Average R2: -0.04517186085069058


**RMSprop**

Результаты для алгоритма RMSprop без регуляризации, с L1 и L2 регуляризациями

In [12]:
method = 'RMSprop'
reg_type = 'No'
k_fold_cross_validation(X, Y, method, reg_type)

Step 1: MSE = 1582.198551534174, R2 = 0.010137619613318938
Step 2: MSE = 1551.9128369044986, R2 = 0.009407788704395181
Step 3: MSE = 1570.2307513628025, R2 = 0.00919718945075465
Step 4: MSE = 1591.2902963023412, R2 = 0.00950287430663832
Step 5: MSE = 1422.3349514382617, R2 = 0.011634464878385264
Average MSE: 1543.5934775084154
Average R2: 0.00997598739069847


In [7]:
method = 'RMSprop_regul'
reg_type = 'L2'
k_fold_cross_validation(X, Y, method, reg_type)

Step 1: MSE = 1665.3200668851127, R2 = -0.04186524751535492
Step 2: MSE = 1633.264455381576, R2 = -0.04251927686484325
Step 3: MSE = 1653.3768685590346, R2 = -0.04326733306147501
Step 4: MSE = 1676.0852702636635, R2 = -0.0432776762799576
Step 5: MSE = 1504.0755348543244, R2 = -0.045166203197355426
Average MSE: 1626.4244391887423
Average R2: -0.04321914738379724


**Adagrad**

Бeз регуляризации Average MSE =1501.9555589185588

L1 Average MSE = 1569.240207568376

L2 Average MSE = 1576.069480001235

**Adam**

Бeз регуляризации Average MSE = 1500.7610984001499

L1 Average MSE = 1576.0684905678836

L2 Average MSE = 1576.069480001235